In [77]:
import pandas as pd
import numpy as np
import json
pd.options.display.max_columns = None

# Start by loading the population data

In [78]:
raw_pop_data = pd.read_csv(r'C:\Users\Evan\Desktop\Capstone 2\Data\Raw Data\NY_population_by_zip_code.csv', header=1)

In [79]:
raw_pop_data.head()

,Id,zipcode,Geography,population,Margin of Error; Total
0,8600000US06390,6390,ZCTA5 06390,230,139
1,8600000US07421,7421,ZCTA5 07421,7351,509
2,8600000US10001,10001,ZCTA5 10001,23947,1738
3,8600000US10002,10002,ZCTA5 10002,77925,2031
4,8600000US10003,10003,ZCTA5 10003,55035,1789


# Get rid of unneccessary columns

In [80]:
pop_data = raw_pop_data.drop(['Id', 'Geography', 'Margin of Error; Total'], axis=1)
pop_data.columns = ['postalcode', 'population']
pop_data.head()

,postalcode,population
0,6390,230
1,7421,7351
2,10001,23947
3,10002,77925
4,10003,55035


# Now load income data

In [81]:
raw_income_data = pd.read_csv(r'C:\Users\Evan\Desktop\Capstone 2\Data\Raw Data\NY_med_income.csv', header=1)

# Get rid of unnecessary columns

In [82]:
income_data = raw_income_data[['Id2', 'Median income (dollars); Estimate; Households']]
income_data.columns = ['postalcode', 'med_income']
income_data['med_income'] = income_data['med_income'].apply(pd.to_numeric, errors='coerce')
income_data.head()

C:\Users\Evan\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,postalcode,med_income
0,6390,150703.0
1,7421,90412.0
2,10001,85221.0
3,10002,35449.0
4,10003,104441.0


# Now load the race data

In [83]:
race_data = pd.read_csv(r'C:\Users\Evan\Desktop\Capstone 2\Data\Raw Data\NY_race_pop_by_zip_code.csv', header=1)
race_data = race_data[['Id2', 
                       'Estimate; Total:', 
                       'Estimate; Population of one race: - White', 
                       'Estimate; Population of one race: - Black or African American', 
                       'Estimate; Population of one race: - American Indian and Alaska Native', 
                       'Estimate; Population of one race: - Asian alone', 
                       'Estimate; Population of one race: - Native Hawaiian and Other Pacific Islander']]
race_data.columns = ['postalcode', 'total', 'white', 'black', 'native', 'asian', 'pacific islander']

races = ['white', 'black', 'native', 'asian', 'pacific islander']

for race in races:
    race_data[race+'_prop'] = race_data[race] / race_data['total']

race_data = race_data.drop(columns=['total', 'white', 'black', 'native', 'asian', 'pacific islander'], axis=1)
race_data.head()

,postalcode,white_prop,black_prop,native_prop,asian_prop,pacific islander_prop
0,6390,0.969565,0.000000,0.000000,0.000000,0.000000
1,7421,0.882737,0.013332,0.023126,0.017549,0.000000
2,10001,0.623168,0.109659,0.002464,0.218900,0.000209
3,10002,0.314431,0.074649,0.006622,0.432146,0.000064
4,10003,0.779159,0.040792,0.002253,0.131662,0.000963


# Load age data

In [84]:
# Load the raw data
age_data = pd.read_csv(r'C:\Users\Evan\Desktop\Capstone 2\Data\Raw Data\NY_age_by_zip_code.csv', header=1)
# Select only important columns
age_data = age_data[['Id2', 
                     'Percent; Estimate; AGE - Under 5 years', 
                     'Percent; Estimate; AGE - 5 to 9 years', 
                     'Percent; Estimate; AGE - 10 to 14 years', 
                     'Percent; Estimate; AGE - 15 to 19 years',
                     'Percent; Estimate; AGE - 20 to 24 years',
                     'Percent; Estimate; AGE - 25 to 29 years',
                     'Percent; Estimate; AGE - 30 to 34 years',
                     'Percent; Estimate; AGE - 35 to 39 years',
                     'Percent; Estimate; AGE - 40 to 44 years',
                     'Percent; Estimate; AGE - 45 to 49 years',
                     'Percent; Estimate; AGE - 50 to 54 years',
                     'Percent; Estimate; AGE - 55 to 59 years',
                     'Percent; Estimate; AGE - 60 to 64 years',
                     'Percent; Estimate; AGE - 65 to 69 years',
                     'Percent; Estimate; AGE - 70 to 74 years',
                     'Percent; Estimate; AGE - 75 to 79 years',
                     'Percent; Estimate; AGE - 80 to 84 years',
                     'Percent; Estimate; AGE - 85 years and over']]
# Rename columns
age_data.columns = ['postalcode', '0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85+']
# select columns to convert to numeric
cols = ['postalcode', '0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85+']
# convert to numeric
age_data = age_data[cols].apply(pd.to_numeric, errors='coerce')
# Combine columns into fewer bins
age_data['0-14'] = age_data.iloc[:, 1:4].sum(axis=1)
age_data['15-29'] = age_data.iloc[:, 4:7].sum(axis=1)
age_data['30-54'] = age_data.iloc[:, 7:12].sum(axis=1)
age_data['55+'] = age_data.iloc[:, 12:19].sum(axis=1)

In [85]:
age_data.head()
age_data_short = age_data[['postalcode', '0-14', '15-29', '30-54', '55+']]
age_data_short.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   postalcode  1795 non-null   int64  
 1   0-14        1795 non-null   float64
 2   15-29       1795 non-null   float64
 3   30-54       1795 non-null   float64
 4   55+         1795 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 70.2 KB


# Merge dataframes

In [86]:
ny_pop_inc = pop_data.merge(income_data, on='postalcode')

In [87]:
ny_census_data = ny_pop_inc.merge(race_data, on='postalcode')

In [88]:
ny_census_data = pd.merge(ny_census_data, age_data_short, on='postalcode')

In [89]:
ny_census_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1795 entries, 0 to 1794
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   postalcode             1795 non-null   int64  
 1   population             1795 non-null   int64  
 2   med_income             1658 non-null   float64
 3   white_prop             1755 non-null   float64
 4   black_prop             1755 non-null   float64
 5   native_prop            1755 non-null   float64
 6   asian_prop             1755 non-null   float64
 7   pacific islander_prop  1755 non-null   float64
 8   0-14                   1795 non-null   float64
 9   15-29                  1795 non-null   float64
 10  30-54                  1795 non-null   float64
 11  55+                    1795 non-null   float64
dtypes: float64(10), int64(2)
memory usage: 182.3 KB


In [90]:
age_data_short.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   postalcode  1795 non-null   int64  
 1   0-14        1795 non-null   float64
 2   15-29       1795 non-null   float64
 3   30-54       1795 non-null   float64
 4   55+         1795 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 70.2 KB


# Load restaurant data

In [91]:
ny_data = pd.read_csv(r'C:\Users\Evan\Desktop\Capstone 2\Data\Raw Data\NYC_Restaurant_Inspection_Results.csv', header=0)

In [92]:
ny_data = ny_data.drop_duplicates(subset=['DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE', 'CUISINE DESCRIPTION'])

In [93]:
ny_data.head()

,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,VIOLATION DESCRIPTION,SCORE,GRADE
0,UNCLE LOUIE G’S ITALIAN ICES & ICE CREAM,Brooklyn,741,UNION STREET,11215.0,7186236668,"Ice Cream, Gelato, Yogurt, Ices",Evidence of mice or live mice present in facil...,22.0,B
1,ARABICA HOOKAH LOUNGE & RESTAURANT INC,Staten Island,4162,VICTORY BLVD,10314.0,7189828888,Middle Eastern,Personal cleanliness inadequate. Outer garment...,12.0,A
2,BOEUF & BUN,Brooklyn,271,KINGSTON AVE,11213.0,7182218900,Jewish/Kosher,Facility not vermin proof. Harborage or condit...,18.0,NaN
3,EL TRI MX RESTAURANT & BAR CORP,Queens,9912,NORTHERN BLVD,11368.0,7184249600,Mexican,“Choking first aid” poster not posted. “Alcoho...,NaN,NaN
4,BAMBOO GARDEN CHINESE RESTAURANT,Queens,14610,LIBERTY AVENUE,11435.0,7182061339,Chinese,Facility not vermin proof. Harborage or condit...,18.0,NaN


In [94]:
len(ny_data['ZIPCODE'].unique())

228

# Group by zip code and calculate the proportion of each cuisine type

In [95]:
cuisine_prop = ny_data.groupby(['ZIPCODE','CUISINE DESCRIPTION']).size() / ny_data.groupby('ZIPCODE').size()
cuisine_prop = cuisine_prop.unstack(level=-1, fill_value=0)
cuisine_prop.reset_index(inplace=True)
cuisine_prop = cuisine_prop.rename(columns = {'ZIPCODE': 'postalcode'})
cuisine_prop.head(10)
restaurants = cuisine_prop
restaurants.head(10)

CUISINE DESCRIPTION,postalcode,Afghan,African,American,Armenian,Asian,Australian,Bagels/Pretzels,Bakery,Bangladeshi,Barbecue,Basque,"Bottled beverages, including water, sodas, juices, etc.",Brazilian,Café/Coffee/Tea,Cajun,Californian,Caribbean,Chicken,Chilean,Chinese,Chinese/Cuban,Chinese/Japanese,Continental,Creole,Creole/Cajun,Czech,Delicatessen,Donuts,Eastern European,Egyptian,English,Ethiopian,Filipino,French,Fruits/Vegetables,German,Greek,Hamburgers,Hawaiian,Hotdogs,Hotdogs/Pretzels,"Ice Cream, Gelato, Yogurt, Ices",Indian,Indonesian,Iranian,Irish,Italian,Japanese,Jewish/Kosher,"Juice, Smoothies, Fruit Salads",Korean,"Latin (Cuban, Dominican, Puerto Rican, South & Central American)",Mediterranean,Mexican,Middle Eastern,Moroccan,Not Listed/Not Applicable,Nuts/Confectionary,Other,Pakistani,Pancakes/Waffles,Peruvian,Pizza,Pizza/Italian,Polish,Portuguese,Russian,Salads,Sandwiches,Sandwiches/Salads/Mixed Buffet,Scandinavian,Seafood,Soul Food,Soups,Soups & Sandwiches,Southwestern,Spanish,Steak,Tapas,Tex-Mex,Thai,Turkish,Vegetarian,Vietnamese/Cambodian/Malaysia
0,10000.0,0.000000,0.000000,0.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,10001.0,0.000000,0.003839,0.312860,0.0,0.019194,0.001919,0.009597,0.009597,0.000000,0.001919,0.0,0.001919,0.000000,0.111324,0.0,0.001919,0.000000,0.001919,0.0,0.015355,0.001919,0.000000,0.003839,0.000000,0.000000,0.0,0.019194,0.021113,0.000000,0.0,0.000000,0.000000,0.000000,0.011516,0.000000,0.000000,0.000000,0.011516,0.001919,0.001919,0.003839,0.005758,0.013436,0.000000,0.000000,0.026871,0.021113,0.036468,0.000000,0.009597,0.099808,0.005758,0.005758,0.007678,0.000000,0.000000,0.000000,0.000000,0.071017,0.0,0.001919,0.003839,0.021113,0.015355,0.000000,0.000000,0.000000,0.015355,0.013436,0.019194,0.000000,0.003839,0.000000,0.001919,0.000000,0.0,0.015355,0.005758,0.001919,0.003839,0.001919,0.0,0.003839,0.000000
2,10002.0,0.000000,0.000000,0.197318,0.0,0.036398,0.005747,0.001916,0.057471,0.001916,0.001916,0.0,0.003831,0.000000,0.078544,0.0,0.000000,0.001916,0.007663,0.0,0.155172,0.000000,0.001916,0.001916,0.001916,0.001916,0.0,0.003831,0.007663,0.005747,0.0,0.001916,0.000000,0.001916,0.026820,0.000000,0.001916,0.005747,0.003831,0.000000,0.000000,0.001916,0.021073,0.007663,0.000000,0.000000,0.001916,0.024904,0.061303,0.000000,0.007663,0.007663,0.011494,0.009579,0.022989,0.005747,0.000000,0.000000,0.000000,0.059387,0.0,0.000000,0.001916,0.049808,0.009579,0.000000,0.001916,0.001916,0.000000,0.007663,0.005747,0.000000,0.005747,0.000000,0.001916,0.000000,0.0,0.013410,0.000000,0.003831,0.001916,0.017241,0.0,0.007663,0.009579
3,10003.0,0.001471,0.000000,0.220588,0.0,0.022059,0.001471,0.002941,0.013235,0.002941,0.002941,0.0,0.010294,0.000000,0.105882,0.0,0.000000,0.000000,0.005882,0.0,0.051471,0.000000,0.001471,0.000000,0.000000,0.000000,0.0,0.004412,0.005882,0.001471,0.0,0.001471,0.001471,0.004412,0.017647,0.001471,0.000000,0.004412,0.007353,0.001471,0.001471,0.001471,0.022059,0.017647,0.001471,0.001471,0.011765,0.048529,0.089706,0.004412,0.022059,0.008824,0.005882,0.017647,0.036765,0.011765,0.001471,0.000000,0.008824,0.055882,0.0,0.001471,0.000000,0.027941,0.010294,0.004412,0.000000,0.002941,0.008824,0.011765,0.007353,0.001471,0.007353,0.000000,0.000000,0.001471,0.0,0.002941,0.001471,0.002941,0.004412,0.019118,0.0,0.016176,0.004412
4,10004.0,0.000000,0.000000,0.297297,0.0

In [96]:
len(restaurants['postalcode'].unique())

227

# Load the geojson zip code data

In [97]:
filepath = r'C:\Users\Evan\Desktop\Capstone 2\Data\Raw Data\nyc_zip_code_tabulation_areas_polygons.geojson'

with open(filepath) as json_file:
    json_data = json.load(json_file)

## Let's find out which zip codes are in the geojson file

In [98]:
json_zips = []

for f in json_data['features']:
    json_zips.append(f['properties']['postalcode'])

In [99]:
len(json_zips)

262

In [100]:
len(set(json_zips))

247

# There are more zip codes in the geojson file than in the restaurant file. Also there are duplicates in the geojson.

## Let's find out which zip codes differ between the two.

In [101]:
restaurants_set = set(restaurants['postalcode'])

In [102]:
for i in range(0, len(json_zips)): 
    json_zips[i] = int(json_zips[i])

json_set = set(json_zips)

In [103]:
restaurants_not_json = restaurants_set.difference(json_set)

json_not_restaurants = json_set.difference(restaurants_set)

In [104]:
restaurants_not_json

{10000.0, 10179.0, 11241.0, 11242.0, 11249.0, 11550.0, 12345.0}

In [105]:
json_not_restaurants

{83,
 10043,
 10045,
 10047,
 10081,
 10096,
 10097,
 10104,
 10110,
 10120,
 10122,
 10152,
 10162,
 10166,
 10175,
 10196,
 10203,
 10259,
 10260,
 10265,
 10270,
 10275,
 10278,
 10286,
 11096,
 11359,
 11451}

# The zip codes do not match exactly so we need to make them match.

## First let's remove duplicates from the json

In [106]:
data = json_data['features']

unique_elements = { each['properties']['postalcode'] : each for each in data }.values()
len(unique_elements)

247

In [107]:
json_data['features'] = list(unique_elements)

In [108]:
len(json_data['features'])

247

# Now I delete rows from the restaurants data that is not in the json.

In [109]:
restaurants = restaurants[~restaurants['postalcode'].isin(list(restaurants_not_json))]

In [110]:
len(restaurants)

220

# Now I delete rows from the json that are not in the restaurants data.

In [111]:
new_features = []
for element in json_data['features']:
    if int(element['properties']['postalcode']) not in list(json_not_restaurants):
        new_features.append(element)

In [112]:
json_data['features'] = new_features

In [113]:
len(json_data['features'])

220

# Now merge the restaurants data with other data useful for mapping

In [114]:
# Convert the postalcode variable to int
restaurants['postalcode'] = restaurants['postalcode'].astype('int')
# Merge the population and income data with the restaurant data
restaurants = restaurants.merge(ny_census_data, how='left', on='postalcode')

C:\Users\Evan\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [115]:
# Calculate the total number of restaurants in each zipcode and merge into the dataframe
total_restaurants = ny_data.groupby('ZIPCODE').size()
total_restaurants = pd.DataFrame(data = {'postalcode':total_restaurants.index, 'total_restaurants':total_restaurants.values}, dtype=np.int64)
restaurants = restaurants.merge(total_restaurants, how='left', on='postalcode')

In [116]:
# Calculate number of restaurants per capita
restaurants['RPC'] = restaurants['total_restaurants'] / restaurants['population']

In [117]:
restaurants.head()

,postalcode,Afghan,African,American,Armenian,Asian,Australian,Bagels/Pretzels,Bakery,Bangladeshi,Barbecue,Basque,"Bottled beverages, including water, sodas, juices, etc.",Brazilian,Café/Coffee/Tea,Cajun,Californian,Caribbean,Chicken,Chilean,Chinese,Chinese/Cuban,Chinese/Japanese,Continental,Creole,Creole/Cajun,Czech,Delicatessen,Donuts,Eastern European,Egyptian,English,Ethiopian,Filipino,French,Fruits/Vegetables,German,Greek,Hamburgers,Hawaiian,Hotdogs,Hotdogs/Pretzels,"Ice Cream, Gelato, Yogurt, Ices",Indian,Indonesian,Iranian,Irish,Italian,Japanese,Jewish/Kosher,"Juice, Smoothies, Fruit Salads",Korean,"Latin (Cuban, Dominican, Puerto Rican, South & Central American)",Mediterranean,Mexican,Middle Eastern,Moroccan,Not Listed/Not Applicable,Nuts/Confectionary,Other,Pakistani,Pancakes/Waffles,Peruvian,Pizza,Pizza/Italian,Polish,Portuguese,Russian,Salads,Sandwiches,Sandwiches/Salads/Mixed Buffet,Scandinavian,Seafood,Soul Food,Soups,Soups & Sandwiches,Southwestern,Spanish,Steak,Tapas,Tex-Mex,Thai,Turkish,Vegetarian,Vietnamese/Cambodian/Malaysia,population,med_income,white_prop,black_prop,native_prop,asian_prop,pacific islander_prop,0-14,15-29,30-54,55+,total_restaurants,RPC
0,10001,0.000000,0.003839,0.312860,0.0,0.019194,0.001919,0.009597,0.009597,0.000000,0.001919,0.0,0.001919,0.0,0.111324,0.0,0.001919,0.000000,0.001919,0.0,0.015355,0.001919,0.000000,0.003839,0.000000,0.000000,0.0,0.019194,0.021113,0.000000,0.0,0.000000,0.000000,0.000000,0.011516,0.000000,0.000000,0.000000,0.011516,0.001919,0.001919,0.003839,0.005758,0.013436,0.000000,0.000000,0.026871,0.021113,0.036468,0.000000,0.009597,0.099808,0.005758,0.005758,0.007678,0.000000,0.000000,0.0,0.000000,0.071017,0.0,0.001919,0.003839,0.021113,0.015355,0.000000,0.000000,0.000000,0.015355,0.013436,0.019194,0.000000,0.003839,0.0,0.001919,0.000000,0.0,0.015355,0.005758,0.001919,0.003839,0.001919,0.0,0.003839,0.000000,23947.0,85221.0,0.623168,0.109659,0.002464,0.218900,0.000209,9.8,28.0,38.3,23.9,521,0.021756
1,10002,0.000000,0.000000,0.197318,0.0,0.036398,0.005747,0.001916,0.057471,0.001916,0.001916,0.0,0.003831,0.0,0.078544,0.0,0.000000,0.001916,0.007663,0.0,0.155172,0.000000,0.001916,0.001916,0.001916,0.001916,0.0,0.003831,0.007663,0.005747,0.0,0.001916,0.000000,0.001916,0.026820,0.000000,0.001916,0.005747,0.003831,0.000000,0.000000,0.001916,0.021073,0.007663,0.000000,0.000000,0.001916,0.024904,0.061303,0.000000,0.007663,0.007663,0.011494,0.009579,0.022989,0.005747,0.000000,0.0,0.000000,0.059387,0.0,0.000000,0.001916,0.049808,0.009579,0.000000,0.001916,0.001916,0.000000,0.007663,0.005747,0.000000,0.005747,0.0,0.001916,0.000000,0.0,0.013410,0.000000,0.003831,0.001916,0.017241,0.0,0.007663,0.009579,77925.0,35449.0,0.314431,0.074649,0.006622,0.432146,0.000064,11.2,20.9,35.1,32.9,522,0.006699
2,10003,0.001471,0.000000,0.220588,0.0,0.022059,0.001471,0.002941,0.013235,0.002941,0.002941,0.0,0.010294,0.0,0.105882,0.0,0.000000,0.000000,0.005882,0.0,0.051471,0.000000,0.001471,0.000000,0.000000,0.000000,0.0,0.004412,0.005882,0.001471,0.0,0.001471,0.001471,0.004412,0.017647,0.001471,0.000000,0.004412,0.007353,0.001471,0.001471,0.001471,0.022059,0.017647,0.001471,0.001471,0.011765,0.048529,0.089706,0.004412,0.022059,0.008824,0.005882,0.017647,0.036765,0.011765,0.001471,0.0,0.008824,0.055882,0.0,0.001471,0.000000,0.027941,0.010294,0.004412,0.000000,0.002941,0.008824,0.011765,0.007353,0.001471,0.007353,0.0,0.000000,0.001471,0.0,0.002941,0.001471,0.002941,0.004412,0.019118,0.0,0.016176,0.004412,55035.0,104441.0,0.779159,0.040792,0.002253,0.131662,0.000963,5.9,37.9,34.3,21.8,680,0.012356
3,10004,0.000000,0.000000,0.297297,0.0,0.013514,0.000000,0.054054,0.006757,0.000000,0.000000,0.0,0.000000,0.0,0.108108,0.0,0.000000,0.000000,0.000000,0.0,0.013514,0.000000,0.000000,0.006757,0.000000,0.000000,0.0,0.006757,0.006757,0.000000,0.0,0.000000,0.000000,0.000000,0.006757,0.000000,0.006757,0.006757,0.020270,0.013514,0.006757,0.006757,0.000000,0.006757,0.000000,0.000000,0.013514,0.040541,0.020270,0.013514,0.02

# Output the data for analysis and further cleaning

In [118]:
with open(r'C:\Users\Evan\Desktop\Capstone 2\Data\Intermediate Data\JSONs\nyc_zip_codes.geojson', 'w') as outfile:
    json.dump(json_data, outfile)

In [119]:
restaurants.to_csv(r'C:\Users\Evan\Desktop\Capstone 2\Data\Intermediate Data\CSVs\restaurants.csv')